# Chess Engine With TensorFlow

## DataSet

In [1]:
import os

files = [file for file in os.listdir("data/pgn") if file.endswith(".pgn")]

len(files)

79

In [2]:
from chess import pgn

def load_pgn(file_path):
    games = []
    with open(file_path,'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

In [4]:
from tqdm import tqdm

LIMIT_OF_FILES = min(len(files),15)
games = []

i = 1
for file in tqdm(files):
    games.extend(load_pgn(f"data/pgn/{file}"))
    if (i>=LIMIT_OF_FILES):
        break
    i += 1


 18%|█▊        | 14/79 [00:06<00:29,  2.20it/s]


In [5]:
len(games)

2582

### Built & Train Model

In [6]:
import numpy as np
from chess import Board
#import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam


In [7]:
def board_to_matrix(board: Board):
    matrix = np.zeros((8,8,12))
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        row, col = divmod(square,8)
        piece_type = piece.piece_type - 1
        piece_color = 0 if piece.color else 6
        matrix[row,col,piece_type+piece_color] = 1
    return matrix
def create_input_for_nn(games):
    X = []
    y = []
    for game in games:
        board = game.board()
        for move in game.mainline_moves():
            X.append(board_to_matrix(board))
            y.append(move.uci())
            board.push(move)
    return X,y
def encode_moves(moves):
    move_to_int = {move: idx for idx, move in enumerate(set(moves))}
    return [move_to_int[move] for move in moves], move_to_int
    

In [8]:
X, y = create_input_for_nn(games)
y, move_to_int = encode_moves(y)
y = to_categorical(y,num_classes=len(move_to_int))
X = np.array(X)


In [9]:
model = Sequential([
    Conv2D(64,(3,3),activation='relu', input_shape = (8,8,12)),
    Conv2D(128,(3,3),activation='relu'),
    Flatten(),
    Dense(256,activation='relu'),
    Dense(len(move_to_int),activation='softmax')
])
model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
model.fit(X,y,epochs=30,validation_split=0.1,batch_size=64)
model.save("model/chess_model.keras")

c:\Users\rajva\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 6, 6, 64)       │         6,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1831)           │       470,567 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,075,943 (4.10 MB)

 Trainable params: 1,075,943 (4.10 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
3040/3040 ━━━━━━━━━━━━━━━━━━━━ 37s 12ms/step - accuracy: 0.0511 - loss: 6.1938 - val_accuracy: 0.1209 - val_loss: 5.3236
Epoch 2/30
3040/3040 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - accuracy: 0.1373 - loss: 4.9997 - val_accuracy: 0.1408 - val_loss: 4.8432
Epoch 3/30
3040/3040 ━━━━━━━━━━━━━━━━━━━━ 42s 14ms/step - accuracy: 0.1738 - loss: 4.3209 - val_accuracy: 0.1514 - val_loss: 4.7114
Epoch 4/30
3040/3040 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - accuracy: 0.2019 - loss: 3.9032 - val_accuracy: 0.1579 - val_loss: 4.6508
Epoch 5/30
3040/3040 ━━━━━━━━━━━━━━━━━━━━ 48s 16ms/step - accuracy: 0.2317 - loss: 3.5805 - val_accuracy: 0.1583 - val_loss: 4.7224
Epoch 6/30
3040/3040 ━━━━━━━━━━━━━━━━━━━━ 55s 18ms/step - accuracy: 0.2600 - loss: 3.3416 - val_accuracy: 0.1596 - val_loss: 4.8042
Epoch 7/30
3040/3040 ━━━━━━━━━━━━━━━━━━━━ 71s 15ms/step - accuracy: 0.2865 - loss: 3.1386 - val_accuracy: 0.1576 - val_loss: 4.9584
Epoch 8/30
3040/3040 ━━━━━━━━━━━━━━━━━━━━ 43s 14ms/step - accuracy: 0.3092 -

In [10]:
from tensorflow.keras.models import load_model
model = load_model("model/chess_model.keras")


In [14]:
int_to_move = dict(zip(move_to_int.values(),move_to_int.keys()))

def predict_next_move(board):
    board_matrix = board_to_matrix(board).reshape(1,8,8,12)
    predictions = model.predict(board_matrix)[0]
    legal_moves = list(board.legal_moves)
    legal_moves_uci = [move.uci() for move in legal_moves]
    sorted_indices = np.argsort(predictions)[::-1]
    for move_index in sorted_indices:
        move = int_to_move[move_index]
        if move in legal_moves_uci:
            return move
    return None
    

In [15]:
board = Board()

In [246]:
print("Board before prediction:")
print(board)

# Predict and make the move
next_move = predict_next_move(board)
board.push_uci(next_move)

# Display the board after prediction
print("\nPredicted move:", next_move)
print("Board after prediction:")
print(board)

Board before prediction:
r . . . . . . .
. . . . . . . .
p . . . . . . .
. . . . . . . .
. . . . . . k .
. . . . . . . .
. . . . . . . K
. R . . . . . .
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

Predicted move: g4h4
Board after prediction:
r . . . . . . .
. . . . . . . .
p . . . . . . .
. . . . . . . .
. . . . . . . k
. . . . . . . .
. . . . . . . K
. R . . . . . .


In [247]:
print(str(pgn.Game.from_board(board)))

[Event "?"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "?"]
[Black "?"]
[Result "1/2-1/2"]

1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-O Be7 6. Re1 b5 7. Bb3 d6 8. c3 O-O 9. h3 h6 10. d4 Re8 11. Nbd2 Bf8 12. Nf1 Bb7 13. Ng3 Na5 14. Bc2 Nc4 15. b3 Nb6 16. Bd2 g6 17. Qc1 Kh7 18. d5 Bg7 19. c4 Nxc4 20. bxc4 bxc4 21. Qb1 g5 22. Nf5 Nh5 23. Ne7 Nf4 24. a4 Rxe7 25. Bb3 cxb3 26. Nd4 Rc8 27. Nf3 Nxd5 28. exd5+ e4 29. Nd4 Re5 30. Qa2 f5 31. Qxb3 Qd7 32. g4 fxg4 33. Qc4 e3 34. Bxe3 Bxd5 35. Qxd5 Qe8 36. hxg4 Qf8 37. Re2 Qe8 38. Qxd6 cxd6 39. Rae1 Qd7 40. Rf1 Rc6 41. a5 d5 42. Rfe1 Rc5 43. Bd2 Qa4 44. f3 Rc7 45. Rf1 Rce7 46. Bxg5 Re4 47. Bh4 Ra7 48. fxe4 Qb4 49. g5 hxg5 50. Bg3 Qc5 51. Ref2 dxe4 52. Kg2 Qxd4 53. Re1 Qxf2+ 54. Bxf2 Kh6 55. Kg1 e3 56. Kf1 e2+ 57. Kxe2 Kg6 58. Ke3 Bf8 59. Rd1 Be7 60. Rd2 Kf5 61. Rb2 Bd8 62. Kd2 Kg6 63. Kd3 Kf5 64. Ke2 Kf4 65. Bg1 g4 66. Kd3 Kg5 67. Kc2 g3 68. Kc1 g2 69. Kb1 Kf6 70. Ra2 Kf5 71. Kb2 Bxa5 72. Kc2 Bb4 73. Kd1 Ba5 74. Rc2 Bb4 75. Rd2 Ba3 76. Ke